In [11]:
import h5py
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

file_path = 'bike_data.h5'

class StandardScaler(object):
    def __init__(self):
        pass

    def transform(self, mean, std, X):
        X = 1. * (X - mean) / std
        return X

    def inverse_transform(self, mean, std, X):
        X = X * std + mean
        return X

def random_walk_matrix(w) -> np.matrix:
    d = np.array(w.sum(1))
    d_inv = np.power(d, -1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat_inv = np.eye(d_inv.shape[0]) * d_inv
    return d_mat_inv.dot(w)
data = []
with h5py.File(file_path, 'r') as f:
    # Đọc dữ liệu từ hai dataset
    bike_pick = f['bike_pick'][:]
    bike_drop = f['bike_drop'][:]


    data1 = np.stack([bike_pick, bike_drop], axis=2)

    mean, std = np.mean(data1), np.std(data1)

    scaler = StandardScaler()
    data1 = scaler.transform(mean, std, data1)
    data.append(data1)


data = np.concatenate(data, axis=1).transpose((0,2,1))
data = data[:-(672+672)]
T, input_dim, N = data.shape
inputs = data.reshape(-1, N)
u, s, v = np.linalg.svd(inputs)

w = np.diag(s[:25]).dot(v[:25,:]).T
graph = cdist(w, w, metric='euclidean')
support = graph * -1 / np.std(graph) ** 2
support = np.exp(support)

support = support - np.identity(support.shape[0])

support = random_walk_matrix(support)
support += np.identity(support.shape[0])


adjacency_path = f"a.csv"
df = pd.DataFrame(support)
df.to_csv(adjacency_path, index=False, header=False, float_format='%.18e')


bike_pick = bike_pick.astype(np.float64)
bike_drop = bike_drop.astype(np.float64)


# Chuyển thành DataFrame
df_pick = pd.DataFrame(bike_pick)
df_drop = pd.DataFrame(bike_drop)

# Ghi ra file CSV
df_pick.to_csv('bike_pick.csv', index=False, header=False, float_format='%.18e')
df_drop.to_csv('bike_drop.csv', index=False, header=False, float_format='%.18e')
